In [22]:
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import re
import time
from outerhtml_lusojornal2015 import outerhtml

# Extract all the href attributes using regular expressions
links = re.findall(r'href="([^"]+)"', outerhtml)

# Filter out entries with 'javascript:void(0)'
filtered_links = [link for link in links if link != 'javascript:void(0)']

# Convert list to set in order to remove repeated links
unique_links_2018 = list(set(filtered_links))

pages=[]

for link in unique_links_2018:
    pages.append(str('https://arquivo.pt'+link))

titles=[]
texts=[]
urls=[]
category_pages=[]

# Initialize an empty dataframe
df = pd.DataFrame(columns=['Title', 'Text', 'URL'])

In [23]:
# Transform page links into frame links
for i in range(len(pages)):
    if(len(pages[i].split("/https")) == 2):
        # link with two https's
        pages[i] = pages[i].split("/https")[0] + "mp_/https" + pages[i].split("/https")[1]
    else:
        # link with one https and one http
        pages[i] = pages[i].split("/http")[0] + "mp_/http" + pages[i].split("/http")[1]

for url in pages:
    print(url)

https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20160208051814mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20180411114505mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20190902111450mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20170219130542mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20160603001817mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20151120092411mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20161103125028mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20180718004736mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20191213000334mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20171210095003mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20181012131329mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/202006230016

In [24]:
len(pages)

21

In [25]:
pages_2015 = []
pages_2016 = []
pages_2017_2018_2019_2020 = []
pages_2020 = []

for page in pages:
    if "wayback/2015" in page:
        pages_2015.append(page)
    elif "wayback/2016" in page:
        pages_2016.append(page)
    elif ("wayback/2017" in page) or ("wayback/2018" in page) or ("wayback/2019" in page):
        pages_2017_2018_2019_2020.append(page)
    else:
        pages_2020.append(page)

for page in pages_2015:
    pages.append(page+'search?q=corrupção')
    pages.append(page+'search?q=governo')
    pages.append(page+'search?q=despesismo')
    pages.append(page+'search?q=Corrupção+Portugal')
    pages.append(page+'search?q=Alerta')

for page in pages_2016:
    pages.append(page+'/search/label/featured')
    pages.append(page+'/search/label/Portugal')
    pages.append(page+'/search/label/Corrupção')
    pages.append(page+'/search/label/Saúde')
    pages.append(page+'/search/label/Insólitos')
    pages.append(page+'/search/label/Refugiados')
    pages.append(page+'/search/label/Portugueses')

for page in pages_2017_2018_2019_2020:
    pages.append(page+'/search/label/featured')
    pages.append(page+'/search/label/Portugal')
    pages.append(page+'/search/label/Corrupção')
    pages.append(page+'/search/label/2017')
    pages.append(page+'/search/label/Saúde')
    pages.append(page+'/search/label/Verão')
    pages.append(page+'/search/label/Insólitos')
    pages.append(page+'/search/label/Refugiados')
    pages.append(page+'/search/label/Portugueses')
    pages.append(page+'/search/label/Viagens')
    pages.append(page+'/search/label/Estudo')

for page in pages:
    print(page)

https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20160208051814mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20180411114505mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20190902111450mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20170219130542mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20160603001817mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20151120092411mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20161103125028mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20180718004736mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20191213000334mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20171210095003mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/20181012131329mp_/http://lusojornal2015.blogspot.pt/
https://arquivo.pt/wayback/202006230016

In [26]:
for url in pages:
    no_more_links = False
    current_url = url
    while(not(no_more_links)):
        # Remove 'amp;' from button link
        try:
            current_url = current_url.split('amp;')[0] + current_url.split('amp;')[1]
        except:
            pass
        # Extract urls
        page = requests.get(current_url)
        soup = bs4(page.text, 'html.parser')
        try:
            link_elements = soup.find_all("div", class_="post-title")
            for link_element in link_elements:
                link = link_element.find("a")
                if link.get("href") not in urls:
                    urls.append(link.get("href"))
                    print(link.get("href"))
            
            # Press button to load more news articles
            button_url = soup.find("a", class_="blog-pager-older-link")['href']            
            current_url = button_url
            
        except:
            # No more new pages with button press
            no_more_links = True

        time.sleep(3)

https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/lula-pode-ser-envenenado-na-prisao.html
https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/russia-planeia-colocar-em-pratica-o.html
https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/os-arquivos-nacionais-dos-eua-revelarao.html
https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/marcelo-rebelo-de-sousa-diz-que-milagre.html
https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/incendio-deflagra-na-trump-tower-em.html
https://arquivo.pt/wayback/20180417085030mp_/https://lusojornal2015.blogspot.pt/2018/04/a-verdadeira-origem-do-hip-hop-nao-esta.html
https://arquivo.pt/wayback/20180417091630mp_/https://lusojornal2015.blogspot.pt/2018/04/legalizar-canabis-nao-porqueentenda.html
https://arquivo.pt/wayback/20180417091630mp_/https://lusojornal2015.blogspot.pt/2018/0

In [27]:
len(urls)

1476

In [97]:
# urls_test = [
#     "https://arquivo.pt/wayback/20180417091630mp_/https://lusojornal2015.blogspot.pt/2018/04/legalizar-canabis-nao-porqueentenda.html"
# ]

#no_more_pages = False
#proxima_check = False
titles=[]
texts=[]
sources=[]
final_urls = []

# Initialize an empty dataframe
df = pd.DataFrame(columns=['Title', 'Text', 'Source', 'URL'])

for url in set(urls):
    print(url)
    page = requests.get(url)
    soup = bs4(page.text, 'html.parser')
    try:
        news_title = soup.find("h3", class_="post-title entry-title").get_text()

        # Split the title into paragraphs
        paragraphs = news_title.split('\n')

        # Remove empty paragraphs
        paragraphs = [paragraph for paragraph in paragraphs if paragraph.strip()]

        reduced_title = '\n'.join(paragraphs)

        news_text = soup.find("div", class_="entry-content").get_text()

        # Split the text into paragraphs
        paragraphs = news_text.split('\n')

        # Remove empty paragraphs
        paragraphs = [paragraph for paragraph in paragraphs if paragraph.strip()]

        #print(paragraphs[-1])
        # print(len(paragraphs[-1].split('Fonte e Fotos:')[0]))
        # print(len(paragraphs[-1]))

        reduced_text = '\n'.join(paragraphs)

        if len(paragraphs[-1].lower().split('fonte e fotos:')[0]) == 0:
            # Fonte e Fotos in the last line
            news_source = paragraphs[-1].lower().split('fonte e fotos:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines = lines[:-1]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].split('fonte e fotos:')[0]) != len(paragraphs[-1]):
            # Fonte e Fotos in the same line as text
            news_source = paragraphs[-1].lower().split('fonte e fotos:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines[-1] = lines[-1].lower().split('fonte e fotos:')[0]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fonte e foto:')[0]) == 0:
            # Fonte e Fotos in the last line
            news_source = paragraphs[-1].lower().split('fonte e foto:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines = lines[:-1]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fonte e foto:')[0]) != len(paragraphs[-1]):
            # Fonte e Fotos in the same line as text
            news_source = paragraphs[-1].lower().split('fonte e fotos:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines[-1] = lines[-1].lower().split('fote e foto:')[0]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fote e foto:')[0]) == 0:
            # Fonte e Fotos in the last line
            news_source = paragraphs[-1].lower().split('fote e foto:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines = lines[:-1]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fote e foto:')[0]) != len(paragraphs[-1]):
            # Fonte e Fotos in the same line as text
            news_source = paragraphs[-1].lower().split('fote e fotos:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines[-1] = lines[-1].lower().split('fote e foto:')[0]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fonte:')[0]) == 0:
            news_source = paragraphs[-1].lower().split('fonte:')[-1].strip()

            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines = lines[:-1]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        elif len(paragraphs[-1].lower().split('fonte:')[0]) != len(paragraphs[-1]):
            news_source = paragraphs[-1].lower().split('fonte:')[-1].strip()
            
            # Split the text into lines
            lines = reduced_text.splitlines()

            # Remove the last line
            lines[-1] = lines[-1].lower().split('fonte')[0]

            # Join the lines back into a string
            reduced_text = '\n'.join(lines)
        else:
            news_source = 'n/a'

        #print(reduced_title)
        print(reduced_text)
        print('Fonte:',news_source)

        titles.append(reduced_title)
        texts.append(reduced_text)
        sources.append(news_source)
        final_urls.append(url)

    except:
        pass

    time.sleep(3)

# Create a new dataframe from the lists
new_data = pd.DataFrame({'Title': titles, 'Text': texts, 'Source': sources, 'URL': final_urls})

# Concatenate the new dataframe with the existing dataframe
df = pd.concat([df, new_data], ignore_index=True)

https://arquivo.pt/wayback/20170803202711mp_/http://lusojornal2015.blogspot.pt/2017/07/portugal-um-pais-de-oportunidades-pelo.html
Ou de oportunistas, corrigindo o apelo internacional para valorização do nosso espólio cultural. Portugal um paraíso para os oportunistas, sim é mesmo este o slogan apropriado.
Grande máquina da Guarda Nacional republicada… vulgo GNR!!!
Fonte: pplware
https://arquivo.pt/wayback/20190902111459mp_/https://lusojornal2015.blogspot.com/2019/06/animais-vao-encolher-25-devido.html
A mão humana na destruição da biodiversidade pode fazer com os animais mais pequenos, que ocupam menos espaço, tenham mais probabilidade de sobreviver do que os animais maiores.
O número de seres humanos tem aumentado exponencialmente e, eventualmente, poderá chegar ao ponto em que o espaço que temos para nós seja cada vez menor e tenhamos que nos expandir para zonas anteriormente ocupadas por animais.
Nesse caso, os animais mais pequenos e que ocupam menos espaço poderão ter mais probab

In [98]:
len(set(urls))

1476

In [102]:
len(final_urls)

939

In [103]:
len(titles)

939

In [104]:
len(texts)

939

In [105]:
df.head()

,Title,Text,Source,URL
0,"Portugal, um país de oportunidades! Pelo menos...","Ou de oportunistas, corrigindo o apelo interna...",pplware,https://arquivo.pt/wayback/20170803202711mp_/h...
1,Animais vão “encolher” 25% devido à destruição...,A mão humana na destruição da biodiversidade p...,zap,https://arquivo.pt/wayback/20190902111459mp_/h...
2,400 carros de luxo para as Águas de Portugal! ...,Mais uma despesa vampiresca do erário público....,apodrecetuga.blogspot.com,https://arquivo.pt/wayback/20150924112605mp_/h...
3,A VERGONHOSA tradição continua!! VERGONHOSO!!!!,"Durante as festas populares em Foios, perto da...",jn,https://arquivo.pt/wayback/20170803171550mp_/h...
4,"""Portugal tornou-se o sonho de consumo de muit...","Entrevista a José Murilo de Carvalho, membro d...",dn,https://arquivo.pt/wayback/20181103051903mp_/h...


In [106]:
df.head()
df.to_csv("lusojornal2015_dataset.csv", index=False)

In [107]:
loaded_df = pd.read_csv("lusojornal2015_dataset.csv")
loaded_df.head()

,Title,Text,Source,URL
0,"Portugal, um país de oportunidades! Pelo menos...","Ou de oportunistas, corrigindo o apelo interna...",pplware,https://arquivo.pt/wayback/20170803202711mp_/h...
1,Animais vão “encolher” 25% devido à destruição...,A mão humana na destruição da biodiversidade p...,zap,https://arquivo.pt/wayback/20190902111459mp_/h...
2,400 carros de luxo para as Águas de Portugal! ...,Mais uma despesa vampiresca do erário público....,apodrecetuga.blogspot.com,https://arquivo.pt/wayback/20150924112605mp_/h...
3,A VERGONHOSA tradição continua!! VERGONHOSO!!!!,"Durante as festas populares em Foios, perto da...",jn,https://arquivo.pt/wayback/20170803171550mp_/h...
4,"""Portugal tornou-se o sonho de consumo de muit...","Entrevista a José Murilo de Carvalho, membro d...",dn,https://arquivo.pt/wayback/20181103051903mp_/h...


In [139]:
for index, (text, source) in enumerate(zip(df['Text'], df['Source'])):
    # Split the text into lines and select the last line
    lines = text.splitlines()
    last_line = lines[-1]
    if len(last_line) < 200 and source == 'n/a':
        print(index, last_line)

# ALTERAR FONTES ERRADAS

31 Leia Também: RACISMO - MULHER MUÇULMANA É EXPULSA DE COMÍCIO DE DONALD TRUMP
47 Leia Também: CONHEÇAM O FEDELHO QUE INSULTOU OS IDOSOS DE PORTUGAL CHAMANDO-OS DE PESTE GRISALHA
49 Fonte. SIC
64 Assim deixamos o relato de um leitor. Seria bom ser verdade, mas, com tanta história que anda por aí, nunca se sabe.
68 Luso Jornal 2015
88 Seria uma mais valia, o aluguer de uma máquina destas quando existe fogos como os que aconteceu este ano.
89 Artigo publicado originalmente na edição impressa da SÁBADO de 23 de Fevereiro de 2012
93 IN http://aventar.eu
102 Fonte. SIC
105 E mais, sem qualquer concurso, tudo em ajuste directo.
117 Fonte e Fotos. Histórias com Valor
121 Leia Também: EX MILITAR DAS FORÇAS ESPECIAIS BRITÂNICAS CHAMADO PARA AJUDAR NAS BUSCAS A PEDRO DIAS
124 Fonte do artigo original: Viriato à Pedrada
125 Aqui estão os bombeiros mais sensuais de Portugal.
131 Pelo menos ficávamos livres desta gente da Troika de uma vez por todos.
140 Concordam com estas despesas?
147 Artigo na

In [108]:
for source in set(df['Source']):
    print(source)

tuga press
cura pela natureza
noticias ao minuto | foto: público
mistérios do mundo
tá bonito
altamente
rius.com.br
bilbiamtengarsada
amulherequemanda.sapo.pt
economico.sapo.pt
tugapress
delas
observador
apodrecetuga (2015)
expresso
sol 2013
idealista.pt
dgs
liliana valente em observador
sic noticias
a delegação de cruz vermelha já esclareceu que nada tem a ver com o evento, tendo sido acionada a ambulância para uma emergência pré-hospitalar.fonte e foto: bombeiros24
ionline
economico.sapo
record.xl.pt
sol
blastingnews
rederegional.com
papo reto live
correio da manhâ
tuga.press
correio da manhã
pplware
blasting news
muito fixe
alegriasempre.com
vip
apodrecetuga (2015)
commonhealth
entornointeligente
angola 24 horas
pt.gatestoneinstitute.org
económico
jn
noticiashub.blogspot.pt
apodrecetuga.blogspot.pt
fundamental
jornal de negócios 2013
apodrecetuga (2014)
tua saúde
deco
paradigmatrix
noticias ao minutos
youtube
paulo pena (texto), miguel manso (fotos, na croácia), sibila lind (vídeo) 